# How to call functions with chat models

Function Calling is a new way to more reliably connect GPT's capabilities with external tools and APIs.
Some of the benefits of using function calling include:

1. **Modular Code:** Enhances code organization and reusability.
2. **New Capabilities:** Expands AI's inherent functionality range.
3. **Performance Optimizations:** Specialized functions boost efficiency.
4. **Interactive Conversations:** Enables dynamic, real-time dialogues.
5. **Simplicity and Ease:** Maintains clean, manageable AI code.
6. **Better Error Handling:** Isolates and addresses issues effectively.
7. **Adaptability:** Facilitates updates without system disruption.


This notebook covers how to use the Chat Completions API in combination with external functions to extend the capabilities of GPT models.

`functions` is an optional parameter in the Chat Completion API which can be used to provide function specifications. The purpose of this is to enable models to generate function arguments which adhere to the provided specifications. Note that the API will not actually execute any function calls. It is up to developers to execute function calls using model outputs.

If the `functions` parameter is provided then by default the model will decide when it is appropriate to use one of the functions. The API can be forced to use a specific function by setting the `function_call` parameter to `{"name": "<insert-function-name>"}`. The API can also be forced to not use any function by setting the `function_call` parameter to `"none"`. If a function is used, the output will contain `"finish_reason": "function_call"` in the response, as well as a `function_call` object that has the name of the function and the generated function arguments.

### Overview

## Gotta do the imports

In [ ]:
Import-Module PowerShellAI -Force
Import-Module PowerShellAI.Functions -Force

### Here is the PowerShell code to get the weather 

https://github.com/chubin/wttr.in is the REST API to be called

In [ ]:
function Get-CurrentWeather {
    <#
        .FunctionDescription
            Gets the current weather for a location
        .ParameterDescription location
            The location to get the weather for
        .ParameterDescription unit
            The unit to get the weather in
    #>
    param (
        [string]$location,
        [ValidateSet('celsius', 'fahrenheit')]
        [string]$unit
    )
    
    $paramUnit ="m"
    if($unit -eq "fahrenheit") {
        $paramUnit = "u"
    }

    Write-Host "Getting the weather for $location in $unit"
    Invoke-RestMethod "https://wttr.in/$($location)?format=4&$paramUnit"
}

### OpenAI Function Spec
Let's use the metadata from the PowerShell function Get-CurrentWeather to create the OpenAI function spec.

In [ ]:
$functions = ConvertFrom-FunctionDefinition (Get-Command Get-CurrentWeather)
$functions | ConvertTo-Json -Depth 10

### Natural Language for Weather

Straight text. This is a GPT prompt. Needs to be in this format to work with the OpenAI REST API.

In [ ]:
$q = 'what is the weather like today, in Greenland?'
# $q = 'what is the weather like today, in Greenland, farenheight?'

$messages = @(
    New-ChatMessageTemplate user $q
)

$messages

### Call GPT

Now you pass in the message and OpenAI function spec and get back the response.

In [ ]:
$chatResponse = Get-ChatCompletion $messages -functions $functions
$chatResponse

### Here's the GPT Response

Key parts: `finish_reason` and `function_call`

Inspecting the `finish_reason`, if it is `function_call`, then we know that the GPT has generated a function call.
Now you can use the `function_call` to call the function.
In this case `Get-CurrentWeather`

In [ ]:
$chatResponse.choices[0] | ConvertTo-Json -Depth 10

### The Integration of the Chat Models with the PowerShell Functions

The following code is the integration of the chat models with the PowerShell functions.

If the `finish_reason` is `function_call` you can grab the function name and arguments from the message.
The `arguments` are JSON. With some PowerShell magic you can convert the JSON to a hashtable.

Now you have the function name and the arguments. using the call operator `&` you can call the function and splat the arguments.

In [ ]:
$target = $chatResponse.choices[0]

if($target.'finish_reason' -eq 'function_call') {
    $functionName = $target.message.'function_call'.name
    $arguments = $target.message.'function_call'.arguments
    $splatParam = $arguments | ConvertFrom-Json -AsHashtable

    & $functionName @splatParam
}

### The End

That's it! You have a PowerShell function `Get-CurrentWeather` and can use it with Natural Language by integration PowerShell and ChatGPT.

This is a very simple example, but you can use it to build more complex functions and integrate them with ChatGPT.

What are you going to add Natural Language to!?

## All the Code

This is all the individual code snippets from the notebook, wrapped in the function `Invoke-AskWeather`.

In [ ]:
function Get-CurrentWeather {
    <#
        .FunctionDescription
            Gets the current weather for a location
        .ParameterDescription location
            The location to get the weather for
        .ParameterDescription format
            The format to get the weather in
    #>
    param (
        [string]$location,
        [ValidateSet('celsius', 'fahrenheit')]
        [string]$unit
    )

    $paramUnit = "m"
    if ($unit -eq "fahrenheit") {
        $paramUnit = "u"
    }

    Write-Host "Getting the weather for $location in $unit"
    Invoke-RestMethod "https://wttr.in/$($location)?format=4&$paramUnit"
}

function Invoke-AskWeather {
        param(
        [Parameter(Mandatory)]
        $q
    )

    $functions = ConvertFrom-FunctionDefinition (Get-Command Get-CurrentWeather)

    $messages = @(
        New-ChatMessageTemplate user $q
    )

    $chatResponse = Get-ChatCompletion $messages -functions $functions
    $target = $chatResponse.choices[0]

    if ($target.'finish_reason' -eq 'function_call') {
        $functionName = $target.message.'function_call'.name
        $arguments = $target.message.'function_call'.arguments
        $splatParam = $arguments | ConvertFrom-Json -AsHashtable

        & $functionName @splatParam
    }
}

In [ ]:
Invoke-AskWeather 'weather in nyc'


In [ ]:
# Notice the misspelling. GPT will correct it.
Invoke-AskWeather 'What is the weather in halfax? not celcius'